In [11]:
!python3 -m pip install -e .

Obtaining file:///Users/andrew/repos/metal-nn-example
  Preparing metadata (setup.py) ... done
  Attempting uninstall: my-extension
    Found existing installation: my-extension 0.0.1
    Uninstalling my-extension-0.0.1:
      Successfully uninstalled my-extension-0.0.1
  Running setup.py develop for my-extension


In [12]:
import timeit
import torch
import my_extension
import torch.mps.profiler as mps_profiler

In [13]:
heightA = 200
widthA = 100

heightB = 100
widthB = 200

a_val = 1.111111
b_val = 2.22222

In [14]:
# two big ish matricies
a = torch.full((heightA, widthB), a_val).to('mps')
b = torch.full((heightA, widthB), b_val).to('mps')

print(f"{a.device}")
print(f"{b.device}")

mps:0
mps:0


In [15]:
# run it once: 
result = my_extension.matrix_multiply(a, b)

print(f"check val {result[0, 0]}")

check val 138.27139282226562


In [5]:
%%timeit -n 10
result = my_extension.matrix_multiply(a, b)

2.3 ms ± 585 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
# now trying a python cpu matrix multiply by hand for comparison. 
import torch

def matrix_multiply_cpu(a, b):
    assert a.shape[1] == b.shape[0], "Incompatible matrix dimensions"
    
    result = torch.zeros(a.shape[0], b.shape[1])
    for i in range(a.shape[0]):
        for j in range(b.shape[1]):
            for k in range(a.shape[1]):
                result[i, j] += a[i, k] * b[k, j]
    return result


In [7]:
# define the tensors and get the device
e = torch.full((heightA, widthA), 1.111111, dtype=torch.float32)
f = torch.full((heightB, widthB), 2.22222, dtype=torch.float32)

print(f"{e.device}")
print(f"{f.device}")

cpu
cpu


In [8]:
# Warm-up
result = matrix_multiply_cpu(e, f)

print(f"check val {result[0, 0]}")

check val 246.91317749023438


In [10]:
%%timeit -n 10

result_cpu = matrix_multiply_cpu(e, f)

KeyboardInterrupt: 

In [6]:
# two big ish matricies
c = torch.full((heightA, widthB), a_val).to('mps')
d = torch.full((heightA, widthB), b_val).to('mps')

print(f"{c.device}")
print(f"{d.device}")

mps:0
mps:0


In [7]:
# run it once
result = c @ d

print(f"check val {result[0, 0]}")

check val 0.0


In [8]:
%%timeit -n 10
result = c @ d

208 µs ± 65.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
# two big ish matricies
e = torch.full((heightA, widthB), a_val)
f = torch.full((heightA, widthB), b_val)

print(f"{e.device}")
print(f"{f.device}")

cpu
cpu


In [10]:
# run it once
result = e @ f

print(f"check val {result[0, 0]}")

check val 4938.3798828125


In [11]:
%%timeit -n 10
result = e @ f

6.43 ms ± 1.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
# try the add tensors
lengthA = 2000
lengthB = 2000

a_val = 1.111111
b_val = 2.22222

In [13]:
# two big ish matricies
a = torch.full((lengthA, 1), a_val).to('mps')
b = torch.full((lengthB, 1), b_val).to('mps')
a = a.flatten()
b = b.flatten() 

print(f"{a.device}")
print(f"{b.device}")

mps:0
mps:0


In [14]:
# run it once: 
result = my_extension.add_tensors(a, b)

print(f"check val {result[0]}")

check val 3.3333311080932617


In [15]:
%%timeit -n 100

result = my_extension.add_tensors(a, b)

The slowest run took 5.86 times longer than the fastest. This could mean that an intermediate result is being cached.
160 µs ± 120 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%%timeit -n 100

result = a + b

The slowest run took 4.80 times longer than the fastest. This could mean that an intermediate result is being cached.
81.1 µs ± 54.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
